# SPC & DOE Optimization
This notebook performs SPC and a DOE-style regression analysis on the provided dataset.

In [ ]:
import pandas as pd
from pathlib import Path
p = Path('..') / 'data' / 'dataset.csv'
df = pd.read_csv(p)
df.head()

In [ ]:
# Generate I-MR chart and capability info
import matplotlib.pyplot as plt
y = df['Yield_percent'].values
plt.figure(figsize=(10,4))
plt.plot(df['Lot_ID'], y, marker='o')
plt.title('I-Chart: Yield %')
plt.show()

In [ ]:
# Capability calculations
USL = 100.0
LSL = 90.0
proc_mean = y.mean()
proc_std = y.std(ddof=1)
Cp = (USL-LSL)/(6*proc_std)
Cpk = min((USL-proc_mean)/(3*proc_std),(proc_mean-LSL)/(3*proc_std))
print('Process Mean, Std, Cp, Cpk:', proc_mean, proc_std, Cp, Cpk)

## DOE (Regression)
The notebook attempts to run a regression to explore main effects and interactions between Temperature, Pressure and Cycle Time.

In [ ]:
try:
    import statsmodels.formula.api as smf
    df2 = df[['Temperature_C','Pressure_kPa','Cycle_Time_sec','Yield_percent']].copy()
    df2['T_c']=(df2['Temperature_C']-df2['Temperature_C'].mean())/df2['Temperature_C'].std()
    df2['P_c']=(df2['Pressure_kPa']-df2['Pressure_kPa'].mean())/df2['Pressure_kPa'].std()
    df2['C_c']=(df2['Cycle_Time_sec']-df2['Cycle_Time_sec'].mean())/df2['Cycle_Time_sec'].std()
    model = smf.ols('Yield_percent ~ T_c * P_c * C_c', data=df2).fit()
    print(model.summary())
except Exception as e:
    print('Statsmodels not available or failed:', e)
